In [1]:
#!pip install plotly pandas numpy scipy yfinance scipy.stats scipy.signal plotly.express


#!python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import plotly.express as px
import yfinance as yf



In [3]:

# Define parameters
symbol = "^GSPC" # S&P 500 index symbol
per = "4y" # 2 years period
inter = "1d" # 1 hour interval
kde_factor = 0.01 # KDE smoothness factor
prominence = 0.25 # Minimum peak prominence
width = 100 # Maximum peak width in points


In [4]:

# Fetch data from yahoo finance
spx = yf.Ticker(symbol)
df = spx.history(period=per, interval=inter)
df = df.loc[df.index > df.index.max() - pd.DateOffset(years=2)] # Filter for last 2 years
df.drop(columns=["Dividends", "Stock Splits","Open", "High", "Low"], inplace=True) # Drop dividends and stock splits columns


In [5]:

# Plot volume profile
fig = px.histogram(df, x="Close", y="Volume", histfunc="sum", nbins=1200)
fig.show()

# Fit KDE to volume profile
kde = stats.gaussian_kde(df["Close"], weights=df["Volume"], bw_method=kde_factor)
x = np.linspace(df["Close"].min(), df["Close"].max(), 1200)
y = kde(x)

# Plot KDE curve and normalized histogram
fig = px.line(x=x, y=y, labels={"x": "Price", "y": "Density"})
fig.add_histogram(x=df["Close"], y=df["Volume"], histfunc="sum", nbinsx=1200, histnorm="probability density", bingroup=None)
fig.show()


In [6]:

# Find peaks in KDE curve
peaks, properties = signal.find_peaks(y, prominence=prominence*(y.max()-y.min()))

# Plot peaks and their properties
fig = px.line(x=x, y=y, labels={"x": "Price", "y": "Density"})
fig.add_scatter(x=x[peaks], y=y[peaks], mode="markers", marker_color="red")
for i in range(len(peaks)):
    fig.add_vline(x=x[int(properties["left_bases"][i])], line_dash="dash", line_color="green")
    fig.add_vline(x=x[int(properties["right_bases"][i])], line_dash="dash", line_color="green")
    #fig.add_hline(y=properties["prominences"][i], line_dash="dash", line_color="blue")
fig.show()


In [7]:

# Print peak prices and densities
print("Peak prices and densities:")
for i in range(len(peaks)):
    print(f"{x[peaks[i]]:.2f}, {y[peaks[i]]:.6f}")


Peak prices and densities:
3676.71, 0.000961
3759.09, 0.000939
3790.63, 0.001341
3821.14, 0.001489
3854.70, 0.001814
3900.48, 0.002725
3992.02, 0.002258
4018.46, 0.001504
4090.68, 0.001769
4119.16, 0.002310
4136.45, 0.003431
4274.78, 0.001483
4374.45, 0.001877
4462.94, 0.001960
4513.80, 0.001662
4566.69, 0.001405
4699.93, 0.001896
